# Векторные представления слов.

In [3]:
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_distances
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

Загрузим данные из тех же файлов.

In [2]:
!pip install gensim

    100% |████████████████████████████████| 22.8MB 48kB/s ta 0:00:015
    100% |████████████████████████████████| 133kB 66kB/s ta 0:00:01
    100% |████████████████████████████████| 4.2MB 99kB/s ta 0:00:011    50% |████████████████                | 2.1MB 38kB/s eta 0:00:56
    100% |████████████████████████████████| 61kB 32kB/s ta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /Users/dariamaximova/Library/Caches/pip/wheels/b1/9e/7d/bb3d3b55c597e72617140a0638c06382a5f17283881eae163e
  Running setup.py bdist_wheel for bz2file ... done
  Stored in directory: /Users/dariamaximova/Library/Caches/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [4]:
sents = open('data/train_pos.out').read().split('\n\n') + \
        open('data/test_pos.out').read().split('\n\n')


Для word2vec'а лучше использовать лемматизированные слова, а для fasttext'а подойдут и сырые слова.

In [5]:
# сохраним целые предложения на будущее
sents_ = []

# не будем учитывать слова короче 4 (так заодно выкинтся все знаки препинания)
lemmas = []
for sent in sents:
    sent_lemmas = []
    for line in sent.split('\n'):
        lemma = line.split('\t')[1]
        if len(lemma) > 3:
            sent_lemmas.append(lemma.lower())
    lemmas.append(sent_lemmas)

raw = []
for sent in sents:
    sent_lemmas = []
    s = []
    for line in sent.split('\n'):
        lemma = line.split('\t')[0]
        s.append(lemma)
        if len(lemma) > 3:
            sent_lemmas.append(lemma.lower())
    raw.append(sent_lemmas)
    sents_.append(s)

In [6]:
len(lemmas)

83148

In [7]:
lemmas[:10]

[['чей-то', 'рука', 'лечь', 'плечо'],
 ['голодать', 'мёрзнуть', 'вокзал', 'дождь', 'унижать', 'бить'],
 ['дальний', 'конец', 'лагерь', 'распространяться', 'аромат', 'жаркое'],
 ['потратить',
  'куча',
  'деньги',
  'мнение',
  'лишь',
  'чтобы',
  'поехать',
  'пригород',
  'встретиться',
  'девочка',
  'весь'],
 ['подчиняться',
  'интуиция',
  'строитель',
  'приблизиться',
  'котлован',
  'который',
  'расчёт',
  'вскоре',
  'должный',
  'быть',
  'заполниться',
  'вода',
  'напоследок',
  'ощупать',
  'каждый',
  'брелок'],
 ['душа', 'попасть'],
 ['один',
  'нежный',
  'впечатлительный',
  'девочка',
  'увидеть',
  'этот',
  'фильм',
  'сойти',
  'начаться',
  'буйный',
  'помешательство',
  'ужас',
  'война'],
 ['присмотреть', 'себя', 'подарок', 'этот', 'дата'],
 ['близко',
  'ночь',
  'дружно',
  'грузиться',
  'собственный',
  'выделенный',
  'нужда',
  'легковой',
  'транспорт',
  'убираться',
  'город'],
 ['подозревать',
  'вышедший',
  'давным-давно',
  'открытка',
  'быть',
 

In [13]:
# посмотрим параметры
?gensim.models.Word2Vec

In [14]:
w2v = gensim.models.Word2Vec(lemmas, size=200,sg=1, max_vocab_size=100000)

In [15]:
# проверим что за модель получилась, посмотрев на близкие слова
w2v.wv.most_similar('семья')

[('спорт', 0.9314305782318115),
 ('учитель', 0.9286872744560242),
 ('родственник', 0.9269474744796753),
 ('традиция', 0.924402117729187),
 ('творчество', 0.9240153431892395),
 ('роль', 0.9228749871253967),
 ('как_правило', 0.9225726127624512),
 ('родина', 0.9207941293716431),
 ('возраст', 0.920457124710083),
 ('семейный', 0.9192546606063843)]

In [6]:
# посмотрим параметры
?gensim.models.FastText

In [16]:
ft = gensim.models.FastText(raw, min_n=3, max_n=6, sg=1, size=100, max_vocab_size=100000)

In [20]:
# проверим что за модель получилась, посмотрев на близкие слова
ft.wv.most_similar('женщина')

[('мужчина', 0.9776095747947693),
 ('жена', 0.971518337726593),
 ('женщине', 0.9657437801361084),
 ('женщину', 0.9579909443855286),
 ('женская', 0.9570773243904114),
 ('женат', 0.9507824778556824),
 ('любимая', 0.9497514367103577),
 ('вина', 0.9495512843132019),
 ('женщин', 0.9483605623245239),
 ('всякая', 0.9473156929016113)]

### Как ни странно, но из одельных векторов слов можно сделать хорошие представления целых предложений.

Для этого просто усредним векторы отдельных векторов.

In [21]:
X = np.zeros((len(lemmas), 200))

for i, sent in enumerate(lemmas):
    try:
        vectors = w2v[sent]
    except (KeyError, ValueError):
        continue
    
    average_vector = np.mean(vectors, axis=0)
    X[i] = average_vector

Можно также искать похожие предложения с помощью косинусной близости (или каких-то других метрик)

In [22]:
sents_[26]

['Со', 'времени', 'ее', 'отъезда', 'прошло', 'три', 'года', '.']

In [24]:
cosine_distances(X[26].reshape(1,-1), X).argsort()

array([[   26, 54970, 57904, ..., 32956, 60599, 41573]])

In [25]:
sents_[54970]

['С', 'тех', 'пор', 'прошло', 'немало', 'времени', '.']

Для наглядности кластеризуем предложения и посмотрим какие получаются кластеры.

In [26]:
km = MiniBatchKMeans(1000, verbose=1)
km.fit(X)

Init 1/3 with method: k-means++
Inertia for init 1/3: 4.577089
Init 2/3 with method: k-means++
Inertia for init 2/3: 4.929391
Init 3/3 with method: k-means++
Inertia for init 3/3: 5.525910
Minibatch iteration 1/83200: mean batch inertia: 0.096289, ewa inertia: 0.096289 
Minibatch iteration 2/83200: mean batch inertia: 0.113559, ewa inertia: 0.096331 
Minibatch iteration 3/83200: mean batch inertia: 0.120676, ewa inertia: 0.096389 
Minibatch iteration 4/83200: mean batch inertia: 0.119075, ewa inertia: 0.096444 
Minibatch iteration 5/83200: mean batch inertia: 0.100133, ewa inertia: 0.096453 
Minibatch iteration 6/83200: mean batch inertia: 0.103523, ewa inertia: 0.096470 
Minibatch iteration 7/83200: mean batch inertia: 0.120630, ewa inertia: 0.096528 
Minibatch iteration 8/83200: mean batch inertia: 0.102404, ewa inertia: 0.096542 
Minibatch iteration 9/83200: mean batch inertia: 0.105247, ewa inertia: 0.096563 
[MiniBatchKMeans] Reassigning 50 cluster centers.
Minibatch iteration 10/

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10,
        n_clusters=1000, n_init=3, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=1)

In [27]:
f = open('clusters_.txt', 'w')

labels = km.labels_ # в km.labels_ тут хранятся кластеры для каждого примера

clusters = defaultdict(list)
for i in range(len(sents_)):
    clusters[labels[i]].append(sents_[i])

for cluster in clusters:
    f.write('\n\n\nCLUSTER - {}\n'.format(cluster))
    for sent in clusters[cluster]:
        f.write(' '.join(sent) + '\n')
f.close()
    

Простое усреднение работает очень хорошо. Если найти корпус побольше и подобрать параметры, векторы предложения будут очень хорошие и подойдут для классификации.

Есть несколько способоб сделать векторы ещё лучше.


Про них можно почитать вот тут:

https://github.com/nlptown/sentence-similarity/blob/master/Simple%20Sentence%20Similarity.ipynb